### This is the third step in running the corridor analysis. 
It splits the resistance raster and hubs into manageable pieces using buffered hexagons. This allows it to run in linkage mapper.

This needs to be run with a map open, so the select at the very end to make the hub outputs will work.

Created by Amy Keister, last run by Amy Keister on August 18, 2023. 

In [1]:
import time
start = time.time()

In [2]:
import os
import arcpy
import numpy

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\C_ConInlandInputPrepHexBuff.gdb"

In [4]:
# define rasters used for cell size, extent, and snapping
SEraster= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"
# I was able to re-snap back this year, but next year I need to use the 90 meter resistance raster as the snap, since we had to resample to get corridors to run
Rextent= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Resistance"

In [5]:
# define additional inputs
Resist= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Resistance"
Hub= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Hubs"
Hex= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Hexagons500000_2023.shp"

# Start Analysis

In [6]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [7]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\C_ConInlandInputPrepHexBuff.gdb


# Buffer Hexagons

In [8]:
arcpy.analysis.Buffer(Hex, "Hex_200kmBuff", "200 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\C_ConInlandInputPrepHexBuff.gdb\\Hex_200kmBuff'>

# Make individual feature classes for each buffered hexagon

In [9]:
# make individual feature classes for each hexagon
arcpy.analysis.SplitByAttributes("Hex_200kmBuff", OutWorkspace, "GRID_ID")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\C_ConInlandInputPrepHexBuff.gdb'>

# Clip raster to buffered hexagons

In [10]:
# make a list of all feature classes
FCList = arcpy.ListFeatureClasses() 

In [11]:
#Print the list made in the above step to see what I found
print(FCList)

['Hex_200kmBuff', 'A_1', 'A_2', 'B_1', 'B_2', 'B_3', 'C_1', 'C_2', 'D_1', 'D_2', 'D_3', 'E_1']


In [12]:
# remove the full buffered hexagon from the list
FCList.remove("Hex_200kmBuff")

In [13]:
#Print the list made in the above step to see what I found
print(FCList)

['A_1', 'A_2', 'B_1', 'B_2', 'B_3', 'C_1', 'C_2', 'D_1', 'D_2', 'D_3', 'E_1']


In [14]:
# Loop to clip resistance raster to buffered hexagons
for fc in FCList:

     output = "Resist_" + fc

     # Process: Buffer
     with arcpy.EnvManager(snapRaster=SEraster):
        out_raster = arcpy.sa.ExtractByMask(Resist, fc); out_raster.save(output)

print ("finished clipping raster") 

finished clipping raster


# select hubs that intersect buffered hexagons

In [19]:
arcpy.management.SelectLayerByLocation(Hub, "INTERSECT", "A_1", None, "NEW_SELECTION", "NOT_INVERT")

<Result 'Hubs_Layer1'>

In [20]:
# Loop to select hubs that intersec the buffered hexagon and export as a new feature class
for fc in FCList:

     output = "Hubs_" + fc

     # Process: Buffer
     with arcpy.EnvManager():
        #arcpy.management.MakeFeatureLayer(Hub, 'Hublyr')
        select = arcpy.management.SelectLayerByLocation(Hub, "INTERSECT", fc, None, "NEW_SELECTION", "NOT_INVERT")
        arcpy.conversion.FeatureClassToFeatureClass(select, OutWorkspace, output) 
        

print ("finished selecting and exporting hubs") 

finished selecting and exporting hubs


In [21]:
end = time.time()
print(end - start)

346.5761706829071
